Script to Clean up Lab Automation Data before Spotfire Import - 

Script 1 is for 1 single file (latest file)

Script 2 is for all unprocessed files in the folder

# Script 1

In [140]:

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import glob
import pathlib


# * means all if need specific format then *.csv
list_of_files = glob.glob('C:/Datafiles/AMC1300/*.txt') 
latest_file = max(list_of_files, key=os.path.getctime)
print (latest_file)
File_name=os.path.basename(latest_file)
print(File_name)
#Remove the .txt extension
File_name = File_name[:-4]

path = pathlib.Path('C:/Datafiles/AMC1300/Cleanedup/'+File_name+'.csv')
re_search = re.search('..Temporary..', File_name)


 
if (path.is_file() == False and re_search == None):
#Read the file
        print("Inside the IF")
        df = pd.read_csv(latest_file,sep='\t',delimiter='\t')
        df=df.set_index('Meas_Name')
        df=df.drop(['RunComment','STRING','TEST_ID'])
        df=df.drop(['Sweep Version', 'Test Area','Package Type','Product Revision','Test_Parameter',
            'Block ID','Channel','Test Type','Start time','Lot','Station Name','SCM',
            'Upper Limit','Lower Limit','Nominal Value','End Time','Execution Duration (ms)',
            'Loop Count','Run Mode Name','UserID','Comment'],axis=1)
        df.to_csv('C:/Datafiles/AMC1300/Cleanedup/'+File_name+'.csv')
     

C:/Datafiles/AMC1300\AMC1311_161558551- Temporary Data File.txt
AMC1311_161558551- Temporary Data File.txt


In [118]:

#for file in list_of_files_processed:
 #   file =os.path.basename(file[:-4])
    #print(file)
    #list_of_files_processed_noext.append(file)



##for file in list_of_files_raw:
 ##   file =file[:-4]
    #print(file)
   ## list_of_files_raw_noext.append(file)
    
##for file in list_of_files_processed:
  ##  file =file[:-4]
    #print(file)
    ##list_of_files_processed_noext.append(file)    
    
##print(list_of_files_raw_noext)  
#print(list_of_files_processed_noext)   


#s = set(list_of_files_processed_noext)
#list_of_files_notprocessed_noext = [x for x in list_of_files_raw_noext if x not in s]
#print(list_of_files_notprocessed_noext)  





# Script 2

In [56]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import glob
import pathlib
import re
list_of_files_raw_noext=[]
list_of_files_raw = glob.glob('C:/Datafiles/AMC1300/*.txt') 
No_of_files_processed=0
No_of_files_skipped=0

for file in list_of_files_raw:
    re_search_bak = re.search('..bak', file)
    
    file_noext =os.path.basename(file[:-4])
    list_of_files_raw_noext.append(file)
    path = pathlib.Path('C:/Datafiles/AMC1300/Cleanedup/'+file_noext+'.csv')
    
    re_search = re.search('..Temporary..', file_noext)
    
    if (path.is_file() == False and re_search==None and re_search_bak==None):
        print("Processing file..."+os.path.basename(file))
         
        #Read the file
        df = pd.read_csv('C:/Datafiles/AMC1300/'+file_noext+'.txt',sep='\t',delimiter='\t')
        
        #set Meas_Name as the DF index
        df=df.set_index('Device Descriptor')
        
        #Rows to be removed (dropped)
        df=df.drop(['DEV_ID','STRING'])
        
        #columns to be removed (dropped)
        df=df.drop(['Sweep Version', 'Test Area','Package Type','Product Revision','Test_Parameter',
            'Block ID','Channel','Test Type','Start time','Lot','Station Name','SCM',
            'Upper Limit','Lower Limit','Nominal Value','End Time','Execution Duration (ms)',
            'Loop Count','Run Mode Name','UserID','Comment'],axis=1)
        
        df = df.rename(columns={col: col.split('(')[0] for col in df.columns})
        #print(df.dtypes)
        
        type_row=[]    
        for column in df:
            #print(column)
            ##print(df[column].unique())
            #print('Column index is',df.columns.get_loc(column))
            Column_idx=df.columns.get_loc(column)
            for unique_val in df[column].unique():
                if(pd.isnull(unique_val)==True):
                #print("this element is NaN")
                  pass
                else:
                    try:
                        float(unique_val)
                        isnumber= True
                        type_row.append('FLOAT')
                        #print(column ,'is a Number')
                        break 
                    except:
                        Isnumber = False   
                        type_row.append('STRING')
                        #print( column,'is a String')
                        break       
        
        
        #Make a data frame with df.columns as index and type_row as column
        df2 = pd.Series(type_row, index=[df.columns])
        #append df2 transposed to df, gets type_as the last row
        df2 = df.append(df2, ignore_index=True)
        #swap last row(type row ) and first row
        last_row, first_row = df2.iloc[-1].copy(), df2.iloc[0].copy()
        df2.iloc[-1],df2.iloc[0] = first_row,last_row
        
        # write cleaned up dataframe to .csv file of the same name
        df2.to_csv('C:/Datafiles/AMC1300/Cleanedup/'+file_noext+'.csv')
        print ('Saved as...'+file_noext+'.csv')
        writer = pd.ExcelWriter('C:/Datafiles/AMC1300/Cleanedup/'+file_noext+'.xlsx')
        df2.to_excel(writer,'Sheet1')
        writer.save()
        print ('Saved as...'+file_noext+'.xlsx')
        #update processed file counter
        No_of_files_processed+=1
       
    else :
        print ('Skipping file...',os.path.basename(file)) 
        No_of_files_skipped+=1     
            
            # import pdb; pdb.set_trace()     # Break point for debug
print(No_of_files_processed,'files processed')  
print (No_of_files_skipped,'files skipped')

df2





Skipping file... AMC1300_Data.txt
Skipping file... AMC1311_133949167.txt
Processing file...AMC1311_13447093.txt
Saved as...AMC1311_13447093.csv
Saved as...AMC1311_13447093.xlsx
Skipping file... AMC1311_141351235.txt
Skipping file... AMC1311_153255936.txt
Skipping file... AMC1311_153356013- Temporary Data File.txt
Skipping file... AMC1311_153356013- Temporary Header File.txt
Skipping file... AMC1311_153656184.txt
Skipping file... AMC1311_153656218.txt
Skipping file... AMC1311_153856315.txt
Skipping file... AMC1311_155557354.txt
Skipping file... AMC1311_155757445.txt
Skipping file... AMC1311_161558551- Temporary Data File.txt
Skipping file... AMC1311_161558551- Temporary Header File.txt
Skipping file... AMC1311_202673595.txt
Skipping file... AMC1311_hdhdhdhdhdh.txt
1 files processed
15 files skipped


,Product Name,Test Program Name,Meas_Name,Meas_Value,Meas_Unit,P/F Status,File Path,DUT,RUN,Temperature,DEVICE_ID_MSB,DEVICE_ID_LSB,VDD2,VDD1,Input_Freq,Vpp,F_Sample,Test_Enable
0,STRING,STRING,STRING,FLOAT,STRING,STRING,STRING,FLOAT,FLOAT,FLOAT,FLOAT,FLOAT,FLOAT,FLOAT,FLOAT,FLOAT,FLOAT,STRING
1,AMC13011,AMC1311_Master_v1_H43E.seq,Timestamp,4.70930000000000000000E+4,NaN,Passed,C:\Datafiles\AMC13011\1\171111130455,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AMC13011,AMC1311_Master_v1_H43E.seq,DUT_ID_LSB_ReadBack,2.00000000000000000000E+1,NaN,Passed,C:\Datafiles\AMC13011\1\171111130455,1.000000,1.000000,125.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AMC13011,AMC1311_Master_v1_H43E.seq,DUT_ID_MSB_ReadBack,4.80000000000000000000E+1,NaN,Passed,C:\Datafiles\AMC13011\1\171111130455,1.000000,1.000000,125.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AMC13011,AMC1311_Master_v1_H43E.seq,Normalized Tone_SigPower,0.00000000000000000000E+0,dB,Passed,C:\Datafiles\AMC13011\1\171111130455,1.000000,1.000000,125.000000,48.000000,20.000000,3.000000,3.000000,1000.000000,0.200000,60000000.000000,Input Bandwidth
5,AMC13011,AMC1311_Master_v1_H43E.seq,Tone_Frequency,1.00002500062501560000E+3,Hz,Passed,C:\Datafiles\AMC13011\1\171111130455,1.000000,1.000000,125.000000,48.000000,20.000000,3.000000,3.000000,1000.000000,0.200000,60000000.000000,Input Bandwidth
6,AMC13011,AMC1311_Master_v1_H43E.seq,Normalized Tone_SigPower,-5.65526748016154100000E-3,dB,Passed,C:\Datafiles\AMC13011\1\171111130455,1.000000,1.000000,125.000000,48.000000,20.000000,3.000000,3.000000,10000.000000,0.200000,60000000.000000,Input Bandwidth
7,AMC13011,AMC1311_Master_v1_H43E.seq,Tone_Frequency,1.00002500062501570000E+4,Hz,Passed,C:\Datafiles\AMC13011\1\171111130455,1.000000,1.000000,125.000000,48.000000,20.000000,3.000000,3.000000,10000.000000,0.200000,60000000.000000,Input Bandwidth
8,AMC13011,AMC1311_Master_v1_H43E.seq,Normalized Tone_SigPower,-3.20311139642797120000E-2,dB,Passed,C:\Datafiles\AMC13011\1\171111130455,1.000000,1.000000,125.000000,48.000000,20.000000,3.000000,3.000000,25000.000000,0.200000,60000000.000000,Input Bandwidth
9,AMC13011,AMC1311_Master_v1_H43E.seq,Tone_Frequency,2.50006250156253920000E+4,Hz,Passed,C:\Datafiles\AMC13011\1\171111130455,1.000000,1.000000,125.000000,48.000000,20.000000,3.000000,3.000000,25000.000000,0.200000,60000000.000000,Input Bandwidth


# All code below is invalid and is used for debug

In [34]:

# if
       # pd.isnull(unique_val)== false :

#df['b'].notnull()
    #print(df[column].notnull())


#print(df.columns.unique())
type_row=[]    
for column in df:
    #print(column)
    ##print(df[column].unique())
    print('Column index is',df.columns.get_loc(column))
    Column_idx=df.columns.get_loc(column)
    for unique_val in df[column].unique():
       if(pd.isnull(unique_val)==True):
          #print("this element is NaN")
           
          pass
       else:
         try:
          #print(pd.to_numeric(df.column.unique(), errors='coerce'))
           float(unique_val)
           isnumber= True
           type_row.append('FLOAT')
           print(column ,'is a Number')
           break 
         except:
          Isnumber = False   
          type_row.append('STRING')
          print( column,'is a String')
          break  


Column index is 0
Device Descriptor is a Number
Column index is 1
Product Name is a String
Column index is 2
Test Program Name is a String
Column index is 3
Meas_Value is a Number
Column index is 4
Meas_Unit is a String
Column index is 5
P/F Status is a String
Column index is 6
File Path is a String
Column index is 7
DUT is a Number
Column index is 8
RUN is a Number
Column index is 9
Temperature is a Number
Column index is 10
DEVICE_ID_MSB is a Number
Column index is 11
DEVICE_ID_LSB is a Number
Column index is 12
VDD2 is a Number
Column index is 13
VDD1 is a Number
Column index is 14
Input_Freq is a Number
Column index is 15
Vpp is a Number
Column index is 16
F_Sample is a Number
Column index is 17
Test_Enable is a String


,Device Descriptor,Product Name,Test Program Name,Meas_Value,Meas_Unit,P/F Status,File Path,DUT,RUN,Temperature,DEVICE_ID_MSB,DEVICE_ID_LSB,VDD2,VDD1,Input_Freq,Vpp,F_Sample,Test_Enable
0,1,AMC13011,AMC1311_Master_v1_H43E.seq,5.33E+04,NaN,NaN,C:\Datafiles\AMC13011\1\171111144840,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,AMC13011,AMC1311_Master_v1_H43E.seq,8.20E+01,NaN,NaN,C:\Datafiles\AMC13011\1\171111144840,1,1,125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,AMC13011,AMC1311_Master_v1_H43E.seq,5.80E+01,NaN,NaN,C:\Datafiles\AMC13011\1\171111144840,1,1,125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,AMC13011,AMC1311_Master_v1_H43E.seq,0.00E+00,dB,Passed,C:\Datafiles\AMC13011\1\171111144840,1,1,125,58,82,3,3,1000,0.2,60000000,Input Bandwidth
4,1,AMC13011,AMC1311_Master_v1_H43E.seq,1.00E+03,Hz,Passed,C:\Datafiles\AMC13011\1\171111144840,1,1,125,58,82,3,3,1000,0.2,60000000,Input Bandwidth
5,1,AMC13011,AMC1311_Master_v1_H43E.seq,-5.18E-03,dB,Passed,C:\Datafiles\AMC13011\1\171111144840,1,1,125,58,82,3,3,10000,0.2,60000000,Input Bandwidth
6,1,AMC13011,AMC1311_Master_v1_H43E.seq,1.00E+04,Hz,Passed,C:\Datafiles\AMC13011\1\171111144840,1,1,125,58,82,3,3,10000,0.2,60000000,Input Bandwidth
7,1,AMC13011,AMC1311_Master_v1_H43E.seq,-3.02E-02,dB,Passed,C:\Datafiles\AMC13011\1\171111144840,1,1,125,58,82,3,3,25000,0.2,60000000,Input Bandwidth
8,1,AMC13011,AMC1311_Master_v1_H43E.seq,2.50E+04,Hz,Passed,C:\Datafiles\AMC13011\1\171111144840,1,1,125,58,82,3,3,25000,0.2,60000000,Input Bandwidth
9,1,AMC13011,AMC1311_Master_v1_H43E.seq,-1.04E-01,dB,Passed,C:\Datafiles\AMC13011\1\171111144840,1,1,125,58,82,3,3,50000,0.2,60000000,Input Bandwidth
